In [2]:
from __future__ import print_function, division
from bs4 import BeautifulSoup
import re
import pandas as pd
import random
from ast import literal_eval #string to dict function


In [2]:
import requests

requests.__path__

['/Users/noramay/anaconda3/lib/python3.7/site-packages/requests']

In [ ]:
#lecture note: Choose model with care. is it fast? slow? interpretable? accurate?

In [3]:
#import imdb dataframe
df = pd.read_csv('title.basics.csv', sep = '\t')

/Users/noramay/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
#create df of just episode information
episode_title_df = df[df['titleType'] == 'tvEpisode']

In [29]:
episode_title_df.sample()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
2960960,tt3957114,tvEpisode,The Rise of the Antichrist,The Rise of the Antichrist,0,2008,\N,76,Reality-TV


In [12]:
#create df of just series information
series_title_df = df[df['titleType'] == 'tvSeries']

In [30]:
series_title_df.sample()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
4566292,tt7503542,tvSeries,Boys Only Club,Boys Only Club,0,2017,\N,\N,Talk-Show


In [ ]:
#df of each episode and corresponding series code 
epi_series_df = pd.read_csv('title.episode.tsv', sep = '\t')

In [31]:
epi_series_df.sample()

,tconst,parentTconst,seasonNumber,episodeNumber
1056502,tt1908832,tt0320037,9,123


In [21]:
rating_df = pd.read_csv('title.ratings.tsv', sep = '\t')

In [34]:
rating_df.sample()

,tconst,averageRating,numVotes
527690,tt1611176,6.4,12


In [24]:
series_title_df.sample(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
397275,tt0413528,tvSeries,Agony,Agony,0,1999,\N,30,\N
4532834,tt7432422,tvSeries,University of the Air,University of the Air,0,1961,1966,30,Documentary
5372078,tt9214736,tvSeries,Trader,Trader,0,\N,\N,\N,\N
2908271,tt3839822,tvSeries,The Son,The Son,0,2017,\N,44,"Drama,Western"
2965318,tt3966654,tvSeries,Greek Lessons for KJV-Only Baptists,Greek Lessons for KJV-Only Baptists,0,2013,\N,\N,Reality-TV
2160028,tt2248752,tvSeries,Stilski izziv,Stilski izziv,0,1998,2002,\N,Talk-Show
2797297,tt3586552,tvSeries,Za dêtaman: Supôtsu no shinjitsu wa suuji ni ari,Za dêtaman: Supôtsu no shinjitsu wa suuji ni ari,0,2013,\N,\N,Sport
3072373,tt4205770,tvSeries,AOL Hardwired 2.0,AOL Hardwired 2.0,0,2014,\N,\N,News
992929,tt1024159,tvSeries,Gala,Gala,0,2002,\N,60,"Music,Talk-Show"
2834033,tt3671376,tvSeries,The Surfers Life,The Surfers Life,0,2008,2009,\N,Sport


In [25]:
epi_series_df[epi_series_df['parentTconst']== 'tt3839822'] #experiment using "The Son"

,tconst,parentTconst,seasonNumber,episodeNumber
2352231,tt5541044,tt3839822,1,1
2472870,tt5924300,tt3839822,1,2
2472872,tt5924306,tt3839822,1,3
2491356,tt5980386,tt3839822,1,4
2491357,tt5980388,tt3839822,1,5
2550914,tt6166012,tt3839822,1,8
2557583,tt6184940,tt3839822,1,6
2557612,tt6185044,tt3839822,1,7
2694239,tt6590564,tt3839822,1,9
2694242,tt6590572,tt3839822,1,10


In [26]:
episode_title_df[episode_title_df['tconst'] == 'tt5541044']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
3669389,tt5541044,tvEpisode,First Son of Texas,First Son of Texas,0,2017,\N,49,"Drama,Western"


In [27]:
rating_df[rating_df['tconst'] == 'tt5541044']

,tconst,averageRating,numVotes
798268,tt5541044,7.6,299


,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


#### Scrape it!

In [447]:
#take a 'title' from imdb dataframe and grab all information from omdb api, if it is a TV series or episode
def get_next_entry(medium, idnum):
    #if medium == 'tvSeries':
    #    url = 'http://www.omdbapi.com/?i={}&apikey=14c936cf'.format(idnum)
    #    response = requests.get(url)
    #    page = response.text
    #   soup = BeautifulSoup(page, "lxml")
    #    soup.prettify();
    #    entryshow = soup.find("p").text
    #    dictshow = literal_eval(entryshow)
    #    print('series')
    #    return dictshow
    if medium == 'tvEpisode':
        url = 'http://www.omdbapi.com/?i={}&apikey=14c936cf'.format(idnum)
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        soup.prettify();
        entryepi = soup.find("p").text
        dictepi = literal_eval(entryepi)
        return dictepi
    else:
        return 0
    

In [221]:
#initialize dataframes with columns from example entries
seriesdf = pd.DataFrame(columns=dict2.keys())
episodedf = pd.DataFrame(columns=dictionary.keys())

In [484]:
#for every entry in csv file, record omdb information into appropriate data frame
#447542
#503782
for i in range(650412,(len(df)-1)):
    print(i)
    dictionary = get_next_entry(df['titleType'][i],df['tconst'][i])
    if dictionary == 0:
        continue
    elif len(dictionary.values()) == 24:
        episodedf = episodedf.append(dictionary, ignore_index =True)
    #elif len(dictionary.values()) == 22:
        #seriesdf = seriesdf.append(dictionary, ignore_index =True)
    else:
        continue


650412
650413
650414
650415
650416
650417
650418
650419
650420
650421
650422
650423
650424
650425
650426
650427
650428
650429
650430
650431
650432
650433
650434
650435
650436
650437
650438
650439
650440
650441
650442
650443
650444
650445
650446
650447
650448
650449
650450
650451
650452
650453
650454
650455
650456
650457
650458
650459
650460
650461
650462
650463
650464
650465
650466
650467
650468
650469
650470
650471
650472
650473
650474
650475
650476
650477
650478
650479
650480
650481
650482
650483
650484
650485
650486
650487
650488
650489
650490
650491
650492
650493
650494
650495
650496
650497
650498
650499
650500
650501
650502
650503
650504
650505
650506
650507
650508
650509
650510
650511
650512
650513
650514
650515
650516
650517
650518
650519
650520
650521
650522
650523
650524
650525
650526
650527
650528
650529
650530
650531
650532
650533
650534
650535
650536
650537
650538
650539
650540
650541
650542
650543
650544
650545
650546
650547
650548
650549
650550
650551
650552
650553
650554

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [458]:
len(df)

5598822

In [432]:

'{"Title":"Three Blind Dates","Year":"2002","Rated":"TV-PG","Released":"05 Mar 2002","Season":"9","Episode":"17","Runtime":"22 min","Genre":"Comedy","Director":"Kelsey Grammer","Writer":"David Angell (created by), Peter Casey (created by), David Lee (created by), Gayle Abrams, Glen Charles (based on the character "Frasier Crane" created by), Les Charles (based on the character "Frasier Crane" created by)","Actors":"N/A","Plot":"It's been a few months since Frasier has been out on a date, so Niles & Daphne, Roz and Martin each take a shot at setting him up with someone. However, perhaps fate is the best matchmaker.","Language":"English","Country":"USA","Awards":"N/A","Poster":"N/A","Ratings":[{"Source":"Internet Movie Database","Value":"7.6/10"}],"Metascore":"N/A","imdbRating":"7.6","imdbVotes":"232","imdbID":"tt0582565","seriesID":"tt0106004","Type":"episode","Response":"True"}
missed:         '
4187202

''

In [477]:
df[df['tconst']=='tt0671588']


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
650183,tt0671588,tvEpisode,Episode #1.1343,Episode #1.1343,0,2004,\N,\N,Drama


In [485]:
episodedf.columns

Index(['Response', 'Error', 'Actors', 'Awards', 'Country', 'Director',
       'Episode', 'Genre', 'Language', 'Metascore', 'Plot', 'Poster', 'Rated',
       'Ratings', 'Released', 'Runtime', 'Season', 'Title', 'Type', 'Writer',
       'Year', 'imdbID', 'imdbRating', 'imdbVotes', 'seriesID'],
      dtype='object')

In [486]:
seriesdf.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'totalSeasons', 'Response'],
      dtype='object')

#### Pickle it!

In [370]:
import pickle

In [483]:
with open('my_data_tvepisode.pkl', 'wb') as picklefile:
    pickle.dump(episodedf, picklefile)

In [438]:
with open('my_data_tvseries.pkl', 'wb') as picklefile:
    pickle.dump(seriesdf, picklefile)

In [312]:
dict2 = get_next_entry(smalldf2['titleType'][1],smalldf2['tconst'][1])

{'Title': 'Heart of a Samurai',
 'Year': '2009–',
 'Rated': 'N/A',
 'Released': '01 Jan 2009',
 'Runtime': 'N/A',
 'Genre': 'Drama',
 'Director': 'N/A',
 'Writer': 'N/A',
 'Actors': 'Satoshi Tsumabuki, Michiko Kichise, Nobuko Miyamoto, Akira Takahashi',
 'Plot': 'N/A',
 'Language': 'Japanese',
 'Country': 'Japan',
 'Awards': 'N/A',
 'Poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BM2I2NDg5ZGUtZjg0OC00NTQxLTg4NGItYjJiMGRjZDI0NjNlXkEyXkFqcGdeQXVyMzE1MjAxNzU@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'}],
 'Metascore': 'N/A',
 'imdbRating': '8.0',
 'imdbVotes': '44',
 'imdbID': 'tt1050624',
 'Type': 'series',
 'totalSeasons': '1',
 'Response': 'True'}

In [235]:
dictionary.keys()

dict_keys(['Title', 'Year', 'Rated', 'Released', 'Season', 'Episode', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'seriesID', 'Type', 'Response'])

24

In [250]:
episodedf = pd.DataFrame(columns=dictionary.keys())
episodedf.loc[0] = list(dictionary.values())
#episodedf.append(list(dictionary.values()))

In [288]:
seriesdf = pd.DataFrame(columns = dict2.keys())

NameError: name 'true' is not defined

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt3772062,tvEpisode,Episode #2.20,Episode #2.20,0,2010,\N,\N,Drama
1,tt1050624,tvSeries,Heart of a Samurai,Tenchijin,0,2009,\N,\N,Drama
2,tt6312422,tvEpisode,Episode #1.147,Episode #1.147,0,1956,\N,\N,"Drama,Romance"
3,tt3542626,tvEpisode,Episode dated 1 October 2013,Episode dated 1 October 2013,0,2013,\N,\N,Talk-Show
4,tt8379312,tvEpisode,Family Business,Family Business,0,2018,\N,20,Comedy
5,tt9431422,tvEpisode,Episode #1.11,Episode #1.11,0,2005,\N,\N,Drama
6,tt0022700,movie,Blondie of the Follies,Blondie of the Follies,0,1932,\N,91,"Comedy,Musical"
7,tt2251325,video,Gaped Crusaders 3,Gaped Crusaders 3,1,2008,\N,182,Adult
8,tt1030971,tvEpisode,Episode dated 24 December 1999,Episode dated 24 December 1999,0,1999,\N,\N,News
9,tt7203406,tvEpisode,James Bond Jr Episode 5 'Plunder Down Under' R...,James Bond Jr Episode 5 'Plunder Down Under' R...,0,2015,\N,\N,Comedy


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
2878158,tt3772062,tvEpisode,Episode #2.20,Episode #2.20,0,2010,\N,\N,Drama
1018374,tt1050624,tvSeries,Heart of a Samurai,Tenchijin,0,2009,\N,\N,Drama
4015197,tt6312422,tvEpisode,Episode #1.147,Episode #1.147,0,1956,\N,\N,"Drama,Romance"
2778651,tt3542626,tvEpisode,Episode dated 1 October 2013,Episode dated 1 October 2013,0,2013,\N,\N,Talk-Show
4977537,tt8379312,tvEpisode,Family Business,Family Business,0,2018,\N,20,Comedy
5475643,tt9431422,tvEpisode,Episode #1.11,Episode #1.11,0,2005,\N,\N,Drama
22340,tt0022700,movie,Blondie of the Follies,Blondie of the Follies,0,1932,\N,91,"Comedy,Musical"
2162301,tt2251325,video,Gaped Crusaders 3,Gaped Crusaders 3,1,2008,\N,182,Adult
999551,tt1030971,tvEpisode,Episode dated 24 December 1999,Episode dated 24 December 1999,0,1999,\N,\N,News
4425457,tt7203406,tvEpisode,James Bond Jr Episode 5 'Plunder Down Under' R...,James Bond Jr Episode 5 'Plunder Down Under' R...,0,2015,\N,\N,Comedy


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


yay


tconst                    tt0000001
titleType                     short
primaryTitle             Carmencita
originalTitle            Carmencita
isAdult                           0
startYear                      1894
endYear                          \N
runtimeMinutes                    1
genres            Documentary,Short
Name: 0, dtype: object

'0.23.4'